In [1]:
import tensorflow as tf
import get
import pandas as pd
import numpy as np
from progressbar import ProgressBar
pbar = ProgressBar()
import tensorflow.keras.layers as layers
from tensorflow.keras import Sequential
import dask.dataframe as dd
from dask.distributed import Client, progress
client = Client(n_workers=2, threads_per_worker=2, memory_limit='8GB')
client

Client Scheduler: tcp://127.0.0.1:55021 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 4 Memory: 16.00 GB


In [2]:
tickers = get.tickers()

In [ ]:
#warning: this takes FOREVER.
data = dict()


for t in pbar(tickers):
    try:
        price_data = get.weekly(t)
        data.update({t:[price_data, get.News(t)]})
    except AttributeError:
        pass

import pickle
with open('stockReg.pkl', 'wb') as f:
    pickle.dump(data, f)    

In [ ]:
#clean data


In [4]:
import pickle
with open('stockReg.pkl', 'rb') as f:
    dataOld = pickle.load(f)

#   prep data

In [5]:
cleanData = dict()

for key, val in dataOld.items():
    if val[0].columns[0] == 'timestamp':
        cleanData.update({key: val})

In [6]:
charts = [v[0] for v in cleanData.values()]

In [55]:
#Only want timestamp and candlestick data, so 
x = np.asarray([c[['timestamp', 'open', 'high', 'low']].to_numpy() for c in charts], dtype = np.float64)
y = np.asarray([c['close'].to_numpy() for c in charts])

ValueError: could not convert string to float: '2019-10-08'

In [43]:
#build Model
stockReg = Sequential()


In [44]:
stockReg.add(layers.Conv1D(100, 4, strides = 5, input_shape = (100, 4)))

In [45]:
stockReg.add(layers.Conv1D(200, 1, strides = 1))

In [46]:
stockReg.add(layers.LSTM(50, use_bias=False ))

In [47]:
stockReg.add(layers.Dense(100, activation = 'tanh'))

In [48]:
stockReg.add(layers.Dense(1, activation= 'tanh'))

In [50]:
stockReg.compile(optimizer = 'Adam', loss= tf.keras.losses.MeanSquaredError())

In [51]:
stockReg.build(input_shape = (100, 1))

In [54]:
stockReg.predict(np.asarray([x[0]], dtype = np.float64), batch_size = 1)

ValueError: could not convert string to float: '2020-03-02'

In [11]:
headlineStatus = Sequential()

In [ ]:
headlineStatus.add(layers.LSTM(100 ))

In [ ]:
headlineStatus.add(layers.Dropout(0.5)

In [ ]:
headlineStatus.add(layers.Dense(50, activation = 'softplus'))
headlineStatus.add(layers.Dense(1, activation = 'tanh'))